In [ ]:
import adal,requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

StatementMeta(sqldwspark, 3, 2, Finished, Available)

In [ ]:
account_name = "azsqldwadls"
container_name= "blob"
adls_path = 'abfss://%s@%s.dfs.core.windows.net/' % (container_name, account_name)

StatementMeta(sqldwspark, 3, 8, Finished, Available)

In [ ]:
groupname="sqldwgroup"

StatementMeta(sqldwspark, 3, 3, Finished, Available)

In [ ]:
def azure_ad_api(url):
    print(url)
    authurl = 'https://login.microsoftonline.com/Tenant_Name/oauth2/v2.0/token'
    data = {
        'grant_type': 'client_credentials',
        'client_id': "20ebb1f6-adf4-459a-8f38-5940d578e9dc",
        'scope': 'https://graph.microsoft.com/.default',
        'client_secret': "client_secret"
             }       
    r = requests.post(authurl, data=data)
    token = r.json().get('access_token')
    headers = {
        'Content-Type' : 'application\json',
        'Authorization': 'Bearer {}'.format(token)
    }
    r = requests.get(url, headers=headers)
    result = r.json()
    return(result)

StatementMeta(sqldwspark, 3, 4, Finished, Available)

In [ ]:

url_group_id="""https://graph.microsoft.com/v1.0/groups?$filter=startswith(displayName, '""" + groupname +  """')&$select=id,displayName"""
result=azure_ad_api(url_group_id)
groupinfo=list(list(result.items())[1])[1]
groupid=str(list(groupinfo[0].values())[0])
url_member_info ='https://graph.microsoft.com/v1.0/groups/'+ groupid + '/transitiveMembers/microsoft.graph.user?$select=userPrincipalName'
result=azure_ad_api(url_member_info)
data=list(list(result.items())[1])[1]
outdf = pd.DataFrame(data)

StatementMeta(sqldwspark, 3, 5, Finished, Available)

https://graph.microsoft.com/v1.0/groups?$filter=startswith(displayName, 'sqldwgroup')&$select=id,displayName
https://graph.microsoft.com/v1.0/groups/3903997c-cda5-46b6-be9c-051e07f708f3/transitiveMembers/microsoft.graph.user?$select=userPrincipalName

In [ ]:
df_sp = spark.createDataFrame(outdf)
df_sp.write.format("parquet").mode("append")\
.option("compression","snappy")\
.save(adls_path +"/"+ "ad_role")

StatementMeta(sqldwspark, 3, 14, Finished, Available)